# S=1

In [1]:
import numpy as np

def site_op(sigma, pos, N):
    I = np.eye(2)
    op = np.eye(1)
    for i in range(1, pos):
        op = np.kron(op, I)
    op = np.kron(op, sigma)
    for i in range(pos+1, N+1):
        op = np.kron(op, I)
    return op

def build_H(M):
    N = M + 2
    sx = np.array([[0,1],[1,0]])
    sz = np.array([[1,0],[0,-1]])
    H = np.zeros((2**N, 2**N))
    for m in range(1, M+1):
        sz_m = site_op(sz, m, N)
        sz_mp1 = site_op(sz, m+1, N)
        sx_mp2 = site_op(sx, m+2, N)
        h_m = sz_m @ sz_mp1 @ sx_mp2
        H += h_m
    return H

def get_epsilon(M):
    H = build_H(M)
    eigs = np.sort(np.linalg.eigvalsh(H))
    epsilon = np.max(np.abs(eigs))
    unique = np.unique(np.round(eigs, decimals=5))
    print(f"For M={M}, unique eigenvalues: {unique}, extracted epsilon = {epsilon}")
    return epsilon

def compute_u(M):
    if M == 1:
        roots = np.roots([-1, 1])  # 1 - x
    elif M == 2:
        roots = np.roots([-2, 1])  # 1 - 2x
    elif M == 3:
        roots = np.roots([-3, 1])  # 1 - 3x
    positive_root = [r for r in roots if r > 0][0]
    u = np.sqrt(positive_root)
    print(f"For M={M}, u^2 = {positive_root}, u = {u}, 1/u = {1/u}")
    return u

def verify(M):
    epsilon = get_epsilon(M)
    u = compute_u(M)
    one_over_u = 1 / u
    print(f"For M={M}, epsilon = {epsilon}, 1/u = {one_over_u}, match: {np.isclose(epsilon, one_over_u, atol=1e-5)}")
    print("\n")

for M in [1,2,3]:
    verify(M)

For M=1, unique eigenvalues: [-1.  1.], extracted epsilon = 1.0
For M=1, u^2 = 1.0, u = 1.0, 1/u = 1.0
For M=1, epsilon = 1.0, 1/u = 1.0, match: True


For M=2, unique eigenvalues: [-1.41421  1.41421], extracted epsilon = 1.4142135623730954
For M=2, u^2 = 0.5, u = 0.7071067811865476, 1/u = 1.414213562373095
For M=2, epsilon = 1.4142135623730954, 1/u = 1.414213562373095, match: True


For M=3, unique eigenvalues: [-1.73205  1.73205], extracted epsilon = 1.7320508075688774
For M=3, u^2 = 0.3333333333333333, u = 0.5773502691896257, 1/u = 1.7320508075688774
For M=3, epsilon = 1.7320508075688774, 1/u = 1.7320508075688774, match: True




# S=2

In [2]:
import numpy as np

# These functions from the previous step are unchanged.
def site_op(sigma, pos, N):
    I = np.eye(2)
    op = np.eye(1)
    for _ in range(1, pos):
        op = np.kron(op, I)
    op = np.kron(op, sigma)
    for _ in range(pos + 1, N + 1):
        op = np.kron(op, I)
    return op

def build_H(M):
    N = M + 2
    sx = np.array([[0, 1], [1, 0]])
    sz = np.array([[1, 0], [0, -1]])
    H = np.zeros((2**N, 2**N))
    for m in range(1, M + 1):
        sz_m = site_op(sz, m, N)
        sz_mp1 = site_op(sz, m + 1, N)
        sx_mp2 = site_op(sx, m + 2, N)
        h_m = sz_m @ sz_mp1 @ sx_mp2
        H += h_m
    return H

# --- New functions for the S=2 case ---

def get_epsilons_from_H(M):
    """
    Calculates the two energy levels, epsilon_1 and epsilon_2,
    by finding the eigenvalues of the Hamiltonian for S=2.
    """
    print(f"For M={M}:")
    H = build_H(M)
    eigs = np.linalg.eigvalsh(H)
    
    # Get unique positive eigenvalues and sort them descending
    positive_eigs = np.unique(np.round(eigs[eigs > 1e-9], decimals=5))
    positive_eigs = np.sort(positive_eigs)[::-1]
    
    print(f"  - Unique positive eigenvalues found: {positive_eigs}")
    
    # Extract the top two eigenvalues
    E_max = positive_eigs[0]
    E_next = positive_eigs[1]
    
    # Solve for epsilon_1 and epsilon_2
    epsilon_1 = (E_max + E_next) / 2
    epsilon_2 = (E_max - E_next) / 2
    
    epsilons = np.sort([epsilon_1, epsilon_2])[::-1]
    print(f"  - Extracted epsilons from H: {epsilons}")
    return epsilons

def get_epsilons_from_u(M):
    """
    Calculates the two expected epsilon values from the roots of P_M(u^2).
    """
    if M == 4:
        # P_4(u^2) = 1 - 4u^2 + (u^2)^2
        # Equation for x=u^2 is: x^2 - 4x + 1 = 0
        poly_coeffs = [1, -4, 1]
    elif M == 5:
        # P_5(u^2) = 1 - 5u^2 + 3(u^2)^2
        # Equation for x=u^2 is: 3x^2 - 5x + 1 = 0
        poly_coeffs = [3, -5, 1]
    elif M == 6:
        # P_6(u^2) = 1 - 6u^2 + 6(u^2)^2
        # Equation for x=u^2 is: 6x^2 - 6x + 1 = 0
        poly_coeffs = [6, -6, 1]
    else:
        raise ValueError("This function is for M=4, 5, 6 only.")

    # Solve for the roots, which are the u_k^2 values
    u_squared_roots = np.roots(poly_coeffs)
    
    # Calculate u_k for each root and then epsilon_k = 1/u_k
    epsilons = [1/np.sqrt(r) for r in u_squared_roots]
    
    # Sort descending to match the convention from the Hamiltonian
    epsilons = np.sort(epsilons)[::-1]
    
    print(f"  - Expected epsilons from P_M(u^2): {epsilons}")
    return epsilons
    
def verify(M):
    """
    Verifies the relation epsilon_k = 1/u_k for S=2.
    """
    epsilons_H = get_epsilons_from_H(M)
    epsilons_u = get_epsilons_from_u(M)
    
    # Use np.allclose to compare the arrays of values
    match = np.allclose(epsilons_H, epsilons_u, atol=1e-5)
    print(f"  - Verification result: Epsilon lists match -> {match}")
    print("-" * 50)

# --- Main Verification Loop ---
for M in [4, 5, 6]:
    verify(M)

For M=4:
  - Unique positive eigenvalues found: [2.44949 1.41421]
  - Extracted epsilons from H: [1.93185 0.51764]
  - Expected epsilons from P_M(u^2): [1.93185165 0.51763809]
  - Verification result: Epsilon lists match -> True
--------------------------------------------------
For M=5:
  - Unique positive eigenvalues found: [2.90931 1.23931]
  - Extracted epsilons from H: [2.07431 0.835  ]
  - Expected epsilons from P_M(u^2): [2.07431329 0.83499962]
  - Verification result: Epsilon lists match -> True
--------------------------------------------------
For M=6:
  - Unique positive eigenvalues found: [3.30136 1.0493 ]
  - Extracted epsilons from H: [2.17533 1.12603]
  - Expected epsilons from P_M(u^2): [2.17532775 1.1260325 ]
  - Verification result: Epsilon lists match -> True
--------------------------------------------------


# S=3

In [3]:
import numpy as np
import time

# These functions are general and remain unchanged.
def site_op(sigma, pos, N):
    I = np.eye(2)
    op = np.eye(1)
    for _ in range(1, pos):
        op = np.kron(op, I)
    op = np.kron(op, sigma)
    for _ in range(pos + 1, N + 1):
        op = np.kron(op, I)
    return op

def build_H(M):
    N = M + 2
    sx = np.array([[0, 1], [1, 0]])
    sz = np.array([[1, 0], [0, -1]])
    H = np.zeros((2**N, 2**N), dtype=np.float64)
    print(f"Building {2**N}x{2**N} matrix for M={M}...")
    for m in range(1, M + 1):
        sz_m = site_op(sz, m, N)
        sz_mp1 = site_op(sz, m + 1, N)
        sx_mp2 = site_op(sx, m + 2, N)
        h_m = sz_m @ sz_mp1 @ sx_mp2
        H += h_m
    return H

# --- New functions for the S=3 case ---

def get_epsilons_from_H(M):
    """
    Calculates the three energy levels for S=3 from the Hamiltonian eigenvalues.
    """
    start_time = time.time()
    H = build_H(M)
    
    print("Finding eigenvalues...")
    eigs = np.linalg.eigvalsh(H)
    
    # Get the 4 unique positive eigenvalues, sorted descending
    positive_eigs = np.unique(np.round(eigs[eigs > 1e-9], decimals=5))
    positive_eigs = np.sort(positive_eigs)[::-1]
    
    print(f"  - Unique positive eigenvalues found: {positive_eigs}")
    
    # Extract the top four eigenvalues
    Ea, Eb, Ec, Ed = positive_eigs[0:4]
    
    # Solve for the three epsilon values
    epsilon_3 = (Ea - Eb) / 2
    epsilon_2 = (Ea - Ec) / 2
    epsilon_1 = Ea - epsilon_2 - epsilon_3
    
    epsilons = np.sort([epsilon_1, epsilon_2, epsilon_3])[::-1]
    
    end_time = time.time()
    print(f"  - Extracted epsilons from H: {epsilons}")
    print(f"  - (Time taken: {end_time - start_time:.2f}s)")
    return epsilons

def get_epsilons_from_u(M):
    """
    Calculates the three expected epsilon values from the roots of P_M(u^2).
    """
    # For np.roots, coeffs are given from highest power to lowest.
    # Let x = u^2.
    if M == 7:
        # P_7(x) = -x^3 + 10x^2 - 7x + 1
        poly_coeffs = [-1, 10, -7, 1]
    elif M == 8:
        # P_8(x) = -4x^3 + 15x^2 - 8x + 1
        poly_coeffs = [-4, 15, -8, 1]
    elif M == 9:
        # P_9(x) = -10x^3 + 21x^2 - 9x + 1
        poly_coeffs = [-10, 21, -9, 1]
    else:
        raise ValueError("This function is for M=7, 8, 9 only.")

    u_squared_roots = np.roots(poly_coeffs)
    epsilons = [1/np.sqrt(r) for r in u_squared_roots]
    epsilons = np.sort(epsilons)[::-1]
    
    print(f"  - Expected epsilons from P_M(u^2): {epsilons}")
    return epsilons
    
def verify(M):
    """
    Verifies the relation epsilon_k = 1/u_k for S=3.
    """
    print(f"--- Verifying for M={M} (S=3) ---")
    epsilons_H = get_epsilons_from_H(M)
    epsilons_u = get_epsilons_from_u(M)
    
    match = np.allclose(epsilons_H, epsilons_u, atol=1e-4) # Increased tolerance slightly for larger matrices
    print(f"  - Verification result: Epsilon lists match -> {match}\n")

# --- Main Verification Loop ---
for M in [7, 8, 9]:
    verify(M)

--- Verifying for M=7 (S=3) ---
Building 512x512 matrix for M=7...
Finding eigenvalues...
  - Unique positive eigenvalues found: [3.93099 3.27358 1.22727 0.56987]
  - Extracted epsilons from H: [2.250425 1.35186  0.328705]
  - (Time taken: 0.11s)
  - Expected epsilons from P_M(u^2): [2.25042986 1.35185795 0.32870284]
  - Verification result: Epsilon lists match -> True

--- Verifying for M=8 (S=3) ---
Building 1024x1024 matrix for M=8...
Finding eigenvalues...
  - Unique positive eigenvalues found: [4.40731 3.2783  1.3362  0.20719]
  - Extracted epsilons from H: [2.30725  1.535555 0.564505]
  - (Time taken: 0.55s)
  - Expected epsilons from P_M(u^2): [2.30725037 1.53555409 0.56450807]
  - Verification result: Epsilon lists match -> True

--- Verifying for M=9 (S=3) ---
Building 2048x2048 matrix for M=9...
Finding eigenvalues...
  - Unique positive eigenvalues found: [4.83346 3.2353  1.46743 0.13073]
  - Extracted epsilons from H: [2.351365 1.683015 0.79908 ]
  - (Time taken: 3.06s)
  -

# S=4

In [4]:
import numpy as np
import time
from itertools import product

# General purpose functions (unchanged)
def site_op(sigma, pos, N):
    I = np.eye(2)
    op = np.eye(1)
    for _ in range(1, pos):
        op = np.kron(op, I)
    op = np.kron(op, sigma)
    for _ in range(pos + 1, N + 1):
        op = np.kron(op, I)
    return op

def build_H(M):
    N = M + 2
    sx = np.array([[0, 1], [1, 0]])
    sz = np.array([[1, 0], [0, -1]])
    # Use float64 for better precision with large matrices
    H = np.zeros((2**N, 2**N), dtype=np.float64)
    print(f"Building {2**N}x{2**N} matrix for M={M}...")
    for m in range(1, M + 1):
        sz_m = site_op(sz, m, N)
        sz_mp1 = site_op(sz, m + 1, N)
        sx_mp2 = site_op(sx, m + 2, N)
        h_m = sz_m @ sz_mp1 @ sx_mp2
        H += h_m
    return H

# --- Functions adapted for S=4 verification ---

def get_hamiltonian_spectrum(M):
    """
    Builds the Hamiltonian matrix and returns its full, sorted spectrum of eigenvalues.
    """
    start_time = time.time()
    H = build_H(M)
    
    print("Finding eigenvalues (this may take some time)...")
    eigs = np.linalg.eigvalsh(H)
    
    end_time = time.time()
    print(f"  - Eigenvalue calculation finished in {end_time - start_time:.2f}s.")
    return np.sort(eigs)

def get_theoretical_spectrum(M, S):
    """
    Calculates the full theoretical spectrum from the roots of P_M(u^2).
    """
    print("Calculating theoretical spectrum from polynomial roots...")
    # For np.roots, coeffs are given from highest power to lowest. Let x = u^2.
    if M == 10:
        # P_10(x) = x^4 - 20x^3 + 28x^2 - 10x + 1
        poly_coeffs = [1, -20, 28, -10, 1]
    elif M == 11:
        # P_11(x) = 5x^4 - 35x^3 + 36x^2 - 11x + 1
        poly_coeffs = [5, -35, 36, -11, 1]
    elif M == 12:
        # P_12(x) = 15x^4 - 56x^3 + 45x^2 - 12x + 1
        poly_coeffs = [15, -56, 45, -12, 1]
    else:
        raise ValueError(f"This function is for M={10, 11, 12} only.")

    # 1. Get base energy levels (epsilons)
    u_squared_roots = np.roots(poly_coeffs)
    epsilons = [1/np.sqrt(r) for r in u_squared_roots]
    
    # 2. Generate all sign combinations for the spectrum
    signs = list(product([-1, 1], repeat=S))
    
    # 3. Calculate all 2^S energy levels
    theoretical_eigs = [np.dot(sign_tuple, epsilons) for sign_tuple in signs]
    
    print(f"  - Base epsilons from roots: {np.sort(epsilons)[::-1]}")
    return np.sort(theoretical_eigs)
    
def verify(M, S):
    """
    Verifies that the Hamiltonian spectrum matches the theoretical spectrum.
    """
    print(f"--- Verifying for M={M} (S={S}) ---")
    
    # Get the spectrum from the Hamiltonian matrix diagonalization
    spec_H = get_hamiltonian_spectrum(M)
    
    # Get the spectrum from the polynomial roots
    spec_theory = get_theoretical_spectrum(M, S)
    
    # The eigenvalues from H will be highly degenerate. We compare the unique values.
    unique_H = np.unique(np.round(spec_H, decimals=4))
    unique_theory = np.unique(np.round(spec_theory, decimals=4))
    
    match = np.allclose(unique_H, unique_theory, atol=1e-4)
    print(f"\n  - Verification result for M={M}: Spectra match -> {match}\n")

# --- Main Verification Loop for S=4 ---
for M in [10, 11, 12]:
    verify(M, S=4)

--- Verifying for M=10 (S=4) ---
Building 4096x4096 matrix for M=10...
Finding eigenvalues (this may take some time)...
  - Eigenvalue calculation finished in 20.16s.
Calculating theoretical spectrum from polynomial roots...
  - Base epsilons from roots: [2.3862093  1.80333036 1.         0.23238932]

  - Verification result for M=10: Spectra match -> True

--- Verifying for M=11 (S=4) ---
Building 8192x8192 matrix for M=11...
Finding eigenvalues (this may take some time)...
  - Eigenvalue calculation finished in 183.64s.
Calculating theoretical spectrum from polynomial roots...
  - Base epsilons from roots: [2.41421356 1.90211303 1.1755705  0.41421356]

  - Verification result for M=11: Spectra match -> True

--- Verifying for M=12 (S=4) ---
Building 16384x16384 matrix for M=12...
Finding eigenvalues (this may take some time)...
  - Eigenvalue calculation finished in 1509.45s.
Calculating theoretical spectrum from polynomial roots...
  - Base epsilons from roots: [2.43703831 1.98410835

In [ ]:
import numpy as np
from functools import lru_cache

def site_op(sigma, pos, N):
    """Creates an operator 'sigma' acting on site 'pos' in a chain of N sites."""
    # This is a faster, more direct way to build the Kronecker product
    op = np.kron(np.eye(2**(pos-1)), sigma)
    op = np.kron(op, np.eye(2**(N-pos)))
    return op

def get_h_m(m, N, sz, sx):
    """Builds the single generator h_m = (sz_m)(sz_m+1)(sx_m+2)."""
    return site_op(sz, m, N) @ site_op(sz, m+1, N) @ site_op(sx, m+2, N)

def build_T(M, u, h_matrices, N):
    """Builds the transfer matrix T_M(u) iteratively."""
    T_cache = {m: np.eye(2**N) for m in range(-2, 1)}
    for m in range(1, M+1):
        T_cache[m] = T_cache[m-1] - u * (h_matrices[m] @ T_cache[m-3])
    return T_cache[M]

def compute_P_val(M, u):
    """Computes the scalar value of the polynomial P_M(u^2) iteratively."""
    P_cache = {m: 1.0 for m in range(-2, 1)}
    u_squared = u**2
    for m in range(1, M+1):
        P_cache[m] = P_cache[m-1] - u_squared * P_cache[m-3]
    return P_cache[M]

def verify_T_relation(M, u_test):
    """Verifies the relation T_M(u) * T_M(-u) = P_M(u^2) * I"""
    print(f"--- Verifying for M = {M} (N={M+2}) at u = {u_test} ---")
    N = M + 2
    sx, sz = np.array([[0,1],[1,0]]), np.array([[1,0],[0,-1]])

    h_matrices = {m: get_h_m(m, N, sz, sx) for m in range(1, M+1)}
    
    # LHS = T(u) @ T(-u)
    T_u = build_T(M, u_test, h_matrices, N)
    T_minus_u = build_T(M, -u_test, h_matrices, N)
    LHS = T_u @ T_minus_u

    # RHS = P_M(u^2) * I
    P_val = compute_P_val(M, u_test)
    RHS = P_val * np.eye(2**N)

    is_match = np.allclose(LHS, RHS, atol=1e-8)
    print(f"Computed P_M(u^2) = {P_val:.6f}, Verification: {is_match}\n")

for M, u_val in [(4, 0.5), (5, 0.3), (8, 0.1)]:
    verify_T_relation(M, u_val)

--- Verifying for M = 4 (N=6) at u = 0.5 ---
Computed P_M(u^2) = 0.062500, Verification: True

--- Verifying for M = 5 (N=7) at u = 0.3 ---
Computed P_M(u^2) = 0.574300, Verification: True

--- Verifying for M = 8 (N=10) at u = 0.1 ---
Computed P_M(u^2) = 0.921496, Verification: True



In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

# --- Helper Functions (Building Blocks) ---

def site_op(sigma, pos, N):
    """Creates an operator 'sigma' acting on site 'pos' in a chain of N sites."""
    op = np.kron(np.eye(2**(pos-1)), sigma)
    op = np.kron(op, np.eye(2**(N-pos)))
    return op

def get_h_m(m, N, sz, sx):
    """Builds the single generator h_m = (sz_m)(sz_m+1)(sx_m+2)."""
    return site_op(sz, m, N) @ site_op(sz, m+1, N) @ site_op(sx, m+2, N)

def build_T(M, u, h_matrices, N):
    """Builds the transfer matrix T_M(u) using the recursion relation."""
    # T_cache stores the computed T_m(u) matrices
    T_cache = {m: np.eye(2**N) for m in range(-2, 1)}

    # Iteratively build T_m(u) up to M
    for m in range(1, M+1):
        T_m = T_cache[m-1] - u * (h_matrices[m] @ T_cache[m-3])
        T_cache[m] = T_m
    
    return T_cache[M] # Return the final transfer matrix T_M(u)

def build_H(M, h_matrices, N):
    """Builds the full Hamiltonian H by summing h_m terms."""
    H = np.zeros((2**N, 2**N))
    for m in range(1, M+1):
        H += h_matrices[m]
    return H

def get_energy_levels(M):
    """
    Finds the energy levels epsilon_k = 1/u_k by finding the
    roots of the polynomial P_M(u^2). (Assumes b_m=1)
    """
    # P_m(x) = P_{m-1}(x) - x * P_{m-3}(x), where x = u^2
    # We use np.poly1d to handle polynomial arithmetic
    P_cache = {m: np.poly1d([1.0]) for m in range(-2, 1)} # P=1
    x_poly = np.poly1d([1.0, 0.0]) # P=x
    
    for m in range(1, M+1):
        P_cache[m] = P_cache[m-1] - x_poly * P_cache[m-3]
    
    P_M = P_cache[M]
    # Roots are the values of x = u^2
    u_squared_roots = P_M.roots
    
    # Filter for positive, real roots and compute u_k and epsilon_k
    u_k_list = []
    for r in u_squared_roots:
        # Check if real and positive (within a small tolerance)
        if np.isreal(r) and r > 1e-9:
            u_k_list.append(np.sqrt(np.real(r)))
            
    # Get unique, sorted lists
    u_k_list = sorted(list(set(u_k_list)))
    epsilon_k_list = sorted([1.0 / u for u in u_k_list])
    
    S_expected = (M + 2) // 3
    if len(u_k_list) != S_expected:
         print(f"Warning: Found {len(u_k_list)} levels, expected S={S_expected}")

    # Return a list of (u_k, epsilon_k) pairs
    return list(zip(u_k_list, epsilon_k_list))

# --- Main Verification Script ---

def verify_Psi_operators(M):
    """
    Runs all 4 tests for the Psi_k operators for a chain of size M.
    """
    print(f"======= Verifying Psi_k Operators for M = {M} (N={M+2}) =======")
    N = M + 2
    S = (M + 2) // 3
    sx = np.array([[0,1],[1,0]])
    sz = np.array([[1,0],[0,-1]])
    I = np.eye(2**N)
    TOL = 1e-8 # Tolerance for np.allclose

    # 1. Pre-compute h_m and build full Hamiltonian H
    print("Building H and h_m matrices...")
    h_matrices = {m: get_h_m(m, N, sz, sx) for m in range(1, M+1)}
    H = build_H(M, h_matrices, N)
    
    # 2. Get energy levels (u_k, epsilon_k)
    print(f"Calculating S = {S} energy level(s)...")
    levels = get_energy_levels(M)
    if not levels:
        print("No positive roots found. Skipping tests.")
        return

    # 3. Build chi_M+1 = sigma_z on site M+2
    chi_M_plus_1 = site_op(sz, M+2, N)

    # Store all Psi operators for Test 4
    all_Psi_ops = {}

    # 4. Loop through each level and run tests
    for k, (u_k, epsilon_k) in enumerate(levels, 1):
        print(f"\n--- Testing level k={k} (u_k={u_k:.4f}, eps_k={epsilon_k:.4f}) ---")

        # Build T(u_k) and T(-u_k)
        T_plus_u = build_T(M, u_k, h_matrices, N)
        T_minus_u = build_T(M, -u_k, h_matrices, N)
        
        # Build Psi_k and Psi_-k (unnormalized)
        Psi_k = T_minus_u @ chi_M_plus_1 @ T_plus_u
        Psi_minus_k = T_plus_u @ chi_M_plus_1 @ T_minus_u
        all_Psi_ops[k] = (Psi_k, Psi_minus_k, epsilon_k)

        # === Test 1: (Psi_k)^2 = 0 ===
        test1_val = Psi_k @ Psi_k
        test1_pass = np.allclose(test1_val, 0, atol=TOL)
        print(f"Test 1 (Pauli Exclusion, Psi_k^2 = 0): {'PASS' if test1_pass else 'FAIL'}")
        if not test1_pass: print(f"  Max error: {np.max(np.abs(test1_val)):.2e}")

        # === Test 2: T(u_k) * Psi_k = 0 ===
        test2_val = T_plus_u @ Psi_k
        test2_pass = np.allclose(test2_val, 0, atol=TOL)
        print(f"Test 2 (T-Matrix Eigenvector, T(u_k)Psi_k = 0): {'PASS' if test2_pass else 'FAIL'}")
        if not test2_pass: print(f"  Max error: {np.max(np.abs(test2_val)):.2e}")
        
        # === Test 3: [H, Psi_k] = 2 * epsilon_k * Psi_k ===
        Comm = H @ Psi_k - Psi_k @ H
        RHS = 2.0 * epsilon_k * Psi_k
        test3_pass = np.allclose(Comm, RHS, atol=TOL)
        print(f"Test 3 (H-Commutator, [H, Psi_k] = 2*eps_k*Psi_k): {'PASS' if test3_pass else 'FAIL'}")
        if not test3_pass: print(f"  Max diff: {np.max(np.abs(Comm - RHS)):.2e}")
    
    # 5. === Test 4: Hamiltonian Reconstruction ===
    print("\n--- Testing H Reconstruction (Test 4) ---")
    # H_recon = sum(eps_k * [Psi_k, Psi_-k])
    H_recon = np.zeros((2**N, 2**N), dtype=complex)
    
    for k in all_Psi_ops:
        Psi_k, Psi_minus_k, epsilon_k = all_Psi_ops[k]
        Comm_k = Psi_k @ Psi_minus_k - Psi_minus_k @ Psi_k
        H_recon += epsilon_k * Comm_k
        
    # H and H_recon should be proportional: H = c * H_recon
    # We find the scaling factor c and check if H = c * H_recon
    H_recon = np.real(H_recon) # H should be real
    
    # Find a non-zero element to get the scaling factor
    idx = np.unravel_index(np.argmax(np.abs(H)), H.shape)
    if np.abs(H_recon[idx]) > 1e-9:
        scale_factor = H[idx] / H_recon[idx]
        RHS = scale_factor * H_recon
        test4_pass = np.allclose(H, RHS, atol=TOL)
        print(f"H_recon = sum(eps_k * [Psi_k, Psi_-k])")
        print(f"Found scaling factor c = {scale_factor:.4f}")
        print(f"Test 4 (H = c * H_recon): {'PASS' if test4_pass else 'FAIL'}")
        if not test4_pass: print(f"  Max diff: {np.max(np.abs(H - RHS)):.2e}")
    else:
        # This will happen if H is zero (M=0) or H_recon is zero
        test4_pass = np.allclose(H, H_recon, atol=TOL) # Check if both are zero
        print(f"Test 4 (H Reconstruction): {'PASS' if test4_pass else 'FAIL'} (Both matrices are near-zero)")
        
    print("="*50 + "\n")

# --- Run Verification ---
# M=3 (S=1 level), M=4 (S=2 levels), M=5 (S=2 levels)
# These are small enough to run quickly.
verify_Psi_operators(M=3)
verify_Psi_operators(M=4)
verify_Psi_operators(M=5)

# M=8 (N=10) involves 1024x1024 matrices and will take a few seconds.
# verify_Psi_operators(M=8)

======= Verifying Psi_k Operators for M = 3 (N=5) =======
Building H and h_m matrices...
Calculating S = 1 energy level(s)...

--- Testing level k=1 (u_k=0.5774, eps_k=1.7321) ---
Test 1 (Pauli Exclusion, Psi_k^2 = 0): PASS
Test 2 (T-Matrix Eigenvector, T(u_k)Psi_k = 0): PASS
Test 3 (H-Commutator, [H, Psi_k] = 2*eps_k*Psi_k): PASS

--- Testing H Reconstruction (Test 4) ---
H_recon = sum(eps_k * [Psi_k, Psi_-k])
Found scaling factor c = 0.1875
Test 4 (H = c * H_recon): PASS

======= Verifying Psi_k Operators for M = 4 (N=6) =======
Building H and h_m matrices...
Calculating S = 2 energy level(s)...

--- Testing level k=1 (u_k=0.5176, eps_k=0.5176) ---
Test 1 (Pauli Exclusion, Psi_k^2 = 0): PASS
Test 2 (T-Matrix Eigenvector, T(u_k)Psi_k = 0): PASS
Test 3 (H-Commutator, [H, Psi_k] = 2*eps_k*Psi_k): FAIL
  Max diff: 2.14e+00

--- Testing level k=2 (u_k=1.9319, eps_k=1.9319) ---
Test 1 (Pauli Exclusion, Psi_k^2 = 0): PASS
Test 2 (T-Matrix Eigenvector, T(u_k)Psi_k = 0): PASS
Test 3 (H-Commut

: 